In [9]:
def compute_area_between_curves(morf_curve, lerf_curve):
    """
    Compute the area between the LeRF and MoRF curves using the trapezoidal rule. In the literature, we divide
    the area by the number of steps to normalize it.
    """
    # Ensure the curves are of the same length
    assert len(morf_curve) == len(lerf_curve), "Curves must be of the same length"

    # Compute the area using the trapezoidal rule
    area = 0.0
    for i in range(1, len(morf_curve)):
        area += 0.5 * ((lerf_curve[i] - morf_curve[i]) + (lerf_curve[i-1] - morf_curve[i-1]))
    return area / len(morf_curve)


def compute_aopc(morf_curve):
    # Compute the Area Over the Perturbation Curve (AOPC)
    reference = morf_curve[0]

    area = 0.0
    for i in range(1, len(morf_curve)):
        area += 0.5 * ((reference - morf_curve[i]) + (reference - morf_curve[i-1]))
    return area / len(morf_curve)

def normalized_abpc(morf_curve, lerf_curve):
    abpc = compute_area_between_curves(morf_curve, lerf_curve)
    range = max(lerf_curve) - min(morf_curve)
    return abpc / range

def normalized_aopc(morf_curve):
    aopc = compute_aopc(morf_curve)
    range = max(morf_curve) - min(morf_curve)
    return aopc / range


In [10]:
# patch_switch_signed_curves.py
# Append swapped/signed curves as `signed_MoRF` / `signed_LeRF` at the same supervoxels_perturbed x,
# and update summary metrics accordingly. Verbose and dry-run by default.

import os
from typing import Dict, List, Optional, Tuple
import numpy as np
import wandb

# ===== USER CONFIG =====
ENTITY = "giuliosichili"
PROJECT = "automi"
GROUP   = "ABPC-volumes"

TARGET_SV_TYPE = "FCC-organs"
TARGET_AGG     = "false_positive_aggregation"

X_KEY     = "supervoxels_perturbed"   # custom x-axis used during logging
PAGE_SIZE = 512

DRY_RUN   = False   # << SAFE DEFAULT: ONLY PRINT, NO WRITES >>
VERBOSE   = True   # << PRINT per-run details >>
# =======================

# Externally provided functions must exist in your environment:
#   compute_area_between_curves(morf_curve, lerf_curve)
#   compute_aopc(morf_curve)
#   normalized_abpc(morf_curve, lerf_curve)
#   normalized_aopc(morf_curve)

def _fmt(x: Optional[float], fixed4: bool = False) -> str:
    if x is None:
        return "None"
    try:
        xf = float(x)
        if np.isnan(xf):
            return "nan"
        return f"{xf:.4f}" if fixed4 else f"{xf:.8f}"
    except Exception:
        return str(x)

def fetch_target_runs(api: wandb.Api) -> List[wandb.apis.public.Run]:
    """Find FCC-organs + false_positive_aggregation runs inside GROUP."""
    path = f"{ENTITY}/{PROJECT}"
    filt = {
        "config.group": {"$eq": GROUP},
        "config.supervoxel_type": {"$eq": TARGET_SV_TYPE},
        "config.aggregation_function": {"$eq": TARGET_AGG},
    }
    return list(api.runs(path, filters=filt))

def collect_curves_with_x_and_last_step(run: wandb.apis.public.Run) -> Tuple[np.ndarray, np.ndarray, np.ndarray, int]:
    """
    Build MoRF/LeRF arrays aligned on the custom x-axis (supervoxels_perturbed).
    Returns (morf, lerf, x_sorted, last_step_seen).
    """
    morf_map: Dict[int, float] = {}
    lerf_map: Dict[int, float] = {}
    last_step_seen = -1

    for row in run.scan_history(page_size=PAGE_SIZE):
        step = row.get("_step", row.get("step"))
        if step is None:
            continue
        try:
            step = int(step)
        except Exception:
            continue
        if step > last_step_seen:
            last_step_seen = step

        # x-axis (prefer supervoxels_perturbed; fallback to step)
        x = row.get(X_KEY, step)
        try:
            x = int(x)
        except Exception:
            try:
                x = int(float(x))
            except Exception:
                continue

        m = row.get("MoRF", None)
        l = row.get("LeRF", None)
        if m is not None:
            try:
                morf_map[x] = float(m)
            except Exception:
                pass
        if l is not None:
            try:
                lerf_map[x] = float(l)
            except Exception:
                pass

    common_x = sorted(set(morf_map.keys()) & set(lerf_map.keys()))
    if not common_x:
        return np.array([]), np.array([]), np.array([]), last_step_seen

    morf = np.array([morf_map[i] for i in common_x], dtype=float)
    lerf = np.array([lerf_map[i] for i in common_x], dtype=float)
    xs   = np.array(common_x, dtype=int)
    return morf, lerf, xs, last_step_seen

def recompute_from_corrected(morf_fixed: np.ndarray, lerf_fixed: np.ndarray) -> Dict[str, float]:
    """Compute corrected metrics from corrected curves using user's functions."""
    out: Dict[str, float] = {}
    out["ABPC_area"] = float(compute_area_between_curves(morf_fixed, lerf_fixed))
    out["AOPC"]      = float(compute_aopc(morf_fixed))
    out["norm_ABPC"] = float(normalized_abpc(morf_fixed, lerf_fixed))
    out["norm_AOPC"] = float(normalized_aopc(morf_fixed))
    return out

def old_summary_metrics(run: wandb.apis.public.Run) -> Dict[str, Optional[float]]:
    s = run.summary or {}
    def g(k):
        v = s.get(k)
        try:
            return float(v) if v is not None else None
        except Exception:
            return None
    return {
        "ABPC_area": g("ABPC_area"),
        "AOPC": g("AOPC"),
        "norm_ABPC": g("norm_ABPC"),
        "norm_AOPC": g("norm_AOPC"),
    }

def main():
    api = wandb.Api()
    runs = fetch_target_runs(api)
    print(f"[target] group='{GROUP}', sv_type='{TARGET_SV_TYPE}', agg='{TARGET_AGG}': {len(runs)} runs")

    patched, skipped = 0, 0
    for run in runs:
        try:
            morf, lerf, xs, last_step = collect_curves_with_x_and_last_step(run)
            if morf.size == 0 or lerf.size == 0:
                if VERBOSE:
                    print(f"- {run.id} SKIP: no aligned MoRF/LeRF on '{X_KEY}'")
                skipped += 1
                continue

            # ---- The actual curve *switch* (swap MoRF/LeRF) ----
            signed_morf = lerf.copy()  # should-be MoRF
            signed_lerf = morf.copy()  # should-be LeRF

            # Recompute corrected metrics from the swapped curves
            new_metrics = recompute_from_corrected(signed_morf, signed_lerf)
            old_metrics = old_summary_metrics(run)

            if VERBOSE:
                print(f"\nRun {run.id} | points={len(xs)} | x in [{xs.min()}, {xs.max()}]")
                print("  ABPC_area : old =", _fmt(old_metrics["ABPC_area"], fixed4=False), " -> new =", _fmt(new_metrics["ABPC_area"], fixed4=False))
                print("  AOPC      : old =", _fmt(old_metrics["AOPC"],      fixed4=False), " -> new =", _fmt(new_metrics["AOPC"],      fixed4=False))
                print("  norm_ABPC : old =", _fmt(old_metrics["norm_ABPC"], fixed4=True),  " -> new =", _fmt(new_metrics["norm_ABPC"], fixed4=True))
                print("  norm_AOPC : old =", _fmt(old_metrics["norm_AOPC"], fixed4=True),  " -> new =", _fmt(new_metrics["norm_AOPC"], fixed4=True))

            if DRY_RUN:
                patched += 1
                continue

            # ---- WRITE (append corrected series + overwrite summary) ----
            os.environ["WANDB_RESUME"] = "allow"
            os.environ["WANDB_RUN_ID"] = run.id
            session = wandb.init(entity=run.entity, project=run.project, id=run.id, resume="allow")

            # Append corrected series at strictly increasing _step; keep the same supervoxels_perturbed
            start_step = int(last_step) + 1
            for i, x in enumerate(xs):
                payload = {
                    "signed_MoRF": float(signed_morf[i]),
                    "signed_LeRF": float(signed_lerf[i]),
                    X_KEY: int(x),  # keep your custom x-axis
                }
                wandb.log(payload, step=start_step + i)

            # Overwrite summary with corrected metrics and store previous for traceability
            session.summary.update({
                "ABPC_area_prev": old_metrics["ABPC_area"],
                "AOPC_prev": old_metrics["AOPC"],
                "norm_ABPC_prev": old_metrics["norm_ABPC"],
                "norm_AOPC_prev": old_metrics["norm_AOPC"],
                **new_metrics,
                "signed_curves": True,
            }, overwrite=True)

            session.finish()
            patched += 1

        except Exception as e:
            if VERBOSE:
                print(f"- {run.id} ERROR: {e}")
            skipped += 1

    print(f"\nDone. Candidate runs: {len(runs)} | Patched (or would patch in DRY_RUN): {patched} | Skipped: {skipped} | DRY_RUN={DRY_RUN} | VERBOSE={VERBOSE}")

if __name__ == "__main__":
    main()

[target] group='ABPC-volumes', sv_type='FCC-organs', agg='false_positive_aggregation': 7 runs

Run yuo4p6ll | points=428 | x in [0, 427]
  ABPC_area : old = 0.00022985  -> new = 0.00022985
  AOPC      : old = 0.00024441  -> new = 0.00024441
  norm_ABPC : old = 0.8218  -> new = 0.8218
  norm_AOPC : old = 0.8738  -> new = 0.8738


- yuo4p6ll ERROR: SummaryDict.update() got an unexpected keyword argument 'overwrite'

Run ci5gxcdn | points=368 | x in [0, 367]
  ABPC_area : old = 0.00039432  -> new = 0.00039432
  AOPC      : old = 0.00042817  -> new = 0.00042817
  norm_ABPC : old = 0.7253  -> new = 0.7253
  norm_AOPC : old = 0.7876  -> new = 0.7876


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


signed_LeRF,███████████████████████████████████▇▇▇▆▁
signed_MoRF,██▇▇▇▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▂▁▂▂▃
supervoxels_perturbed,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇██
ABPC_area,0.00023
ABPC_area_prev,-0.00023
AOPC,0.00024
AOPC_prev,1e-05
LeRF,-0.00025
LeRF_cache_hit_ratio_percent,12.5
LeRF_inference_time_sec,3.77184
LeRF_volume_removed_mm3,6718742.56134


- ci5gxcdn ERROR: SummaryDict.update() got an unexpected keyword argument 'overwrite'

Run mherfqjc | points=339 | x in [0, 338]
  ABPC_area : old = 0.00042865  -> new = 0.00042865
  AOPC      : old = 0.00046886  -> new = 0.00046886
  norm_ABPC : old = 0.7793  -> new = 0.7793
  norm_AOPC : old = 0.8524  -> new = 0.8524


...

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


signed_LeRF,███████████████████▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▄▄▁
signed_MoRF,▃▃▂▃▃▃▂▂▂▁▁▁▁▂▂▂▂▂▁▁▁▂▂▂▂▂▃▅▅▅▅▅▅▅▆▅▅▅▅█
supervoxels_perturbed,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
ABPC_area,0.00033
ABPC_area_prev,-0.00033
AOPC,0.0004
AOPC_prev,7e-05
LeRF,-0.00027
LeRF_cache_hit_ratio_percent,11
LeRF_inference_time_sec,4.89515
LeRF_volume_removed_mm3,8309270.59855


- b10nvc1l ERROR: SummaryDict.update() got an unexpected keyword argument 'overwrite'

Run o0hbjeqc | points=372 | x in [0, 371]
  ABPC_area : old = 0.00018703  -> new = 0.00018703
  AOPC      : old = 0.00021942  -> new = 0.00021942
  norm_ABPC : old = 0.5783  -> new = 0.5783
  norm_AOPC : old = 0.6784  -> new = 0.6784


signed_LeRF,█▇▇▇▇▅▅▅▅▅▅▅▄▄▃▃▃▃▄▄▄▄▃▃▃▃▃▂▃▃▃▃▂▁▁▂▂▁▁▁
signed_MoRF,█▃▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▁▂▂▂▃▃▄▄
supervoxels_perturbed,▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇█████
ABPC_area,0.00023
ABPC_area_prev,-0.00023
AOPC,0.0003
AOPC_prev,7e-05
LeRF,-0.00015
LeRF_cache_hit_ratio_percent,20.83333
LeRF_inference_time_sec,5.09036
LeRF_volume_removed_mm3,3799992.37061


- o0hbjeqc ERROR: SummaryDict.update() got an unexpected keyword argument 'overwrite'

Done. Candidate runs: 7 | Patched (or would patch in DRY_RUN): 0 | Skipped: 7 | DRY_RUN=False | VERBOSE=True
